# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

План работы:
1. Открытые файла с данными и изучение информации
2. Предобработка данных: работа с пропусками
3. Замена типов данных 
4. Обработка дубликатов
5. Категоризация данных
6. Ответы на вопросы:
Есть ли зависимость между наличием детей и возвратом кредита в срок?
Есть ли зависимость между семейным положением и возвратом кредита в срок?
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Как разные цели кредита влияют на его возврат в срок?
7. Выводы и рекомендации


## Смотрим файл с данными и изучим общую информацию. 

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv') #читаем файл csv
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

В таблице 12 столбцов, представленных в трех типах данных: object, float, int. 
Количество значений в стоблцах отличается, значит в столбцах (total_income и days_employed) есть пустые значения.
В days_emloyed и children есть отрицательные значения.
В столбце образование - разный регистр. 

## Предобработка данных

### Обработка пропусков

In [3]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
#проверяю сколько уникальных значений в столбце income_type
display(df.income_type.unique()) #всего 8 категорий
median_incomes = df.groupby('income_type')['total_income'].median().to_dict()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [5]:
def filler_func(row):
    if pd.isna(row['total_income']):
        return median_incomes.get(row['income_type'])
    return row['total_income']
df['total_income'] = df.apply(filler_func, axis=1)
median_days = df.groupby('income_type')['days_employed'].median().to_dict()

In [6]:
def filler_func(row):
    if pd.isna(row['days_employed']):
        return median_incomes.get(row['income_type'])
    return row['days_employed']
df['days_employed'] = df.apply(filler_func, axis=1)

In [7]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Одинаковое количество пропусков в days_employes и total_income. Возможно эти люди вообще не работают или не работали по тем или иным причинам. Либо работали неофициально
В столбце количество детей есть значение 20. Скорее всего опечатка. Только если это не федеральная программа для многодетных
В days_employed значения можно привести к целым числам
В столбце gender есть одно значение XNA


### Замена типа данных

In [8]:
#посмотрим какие типы данных встречаются в таблице
df.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [9]:
# заменим отрицательные значения в столбце days_employed Это значение не может быть отрицательным. Скорее всего опечатки
df['days_employed'] = df['days_employed'].abs()

In [10]:
# заменим отрицательные значения в столбце children Это значение не может быть отрицательным. Скорее всего опечатки.
# также как и количество детей 20. Скорее всего человеческий фактор. Заменю на 2
df['children'] = df['children'].abs()
df['children'] = df['children'].replace(20, 2)

In [11]:
# в таблице был возраст 0. посмотрим что вообще с возрастом
df['dob_years'].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [12]:
# считаю количество людей с недостаточным возрастом для получения кредита (до 18 лет) 
df[df['days_employed'] > 70].groupby(['dob_years'])['dob_years'].count()
df[df['dob_years'] == 0]['dob_years'].count()

101

In [13]:
# в столбце образование и семейное положение нужно привести данные к одному регистру 
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

In [14]:
# в столбце пол было непонятное значение XNA. Что это? или это по-современному "не определился?"
df[df['gender'] == 'XNA'].count()

children            1
days_employed       1
dob_years           1
education           1
education_id        1
family_status       1
family_status_id    1
gender              1
income_type         1
debt                1
total_income        1
purpose             1
dtype: int64

In [15]:
# в столбце days_employed можно привести числа к целым значениям. Для рассчета кредитоспособности погрешность будет небольшая
# а работать с целыми числами удобнее
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

In [16]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


### Вывод

В таблице были некорректные данные. Удалили пропуски, отрицательные значения в столбцах дети, рабочий стаж.

### Обработка дубликатов

In [17]:
df.duplicated().sum()

71

In [18]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [19]:
df.head(10)
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21444,1,467,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486,заняться образованием
21445,0,914,42,высшее,0,женат / замужем,0,F,компаньон,0,322807,покупка своего жилья
21446,0,404,42,высшее,0,гражданский брак,1,F,компаньон,0,178059,на покупку своего автомобиля
21447,0,373995,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем
21448,1,2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля
21453,2,1984,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047,на покупку автомобиля


### Вывод

Удалили 54 дубликата


### Лемматизация

In [20]:
from pymystem3 import Mystem
m = Mystem()
# в словарной форме данные представлены в столбцах: образование, семейное положение, статус и цель кредита.
# со столбцами образование, семейное положение и статус все в порядке.
#смотрю на какие же цели люди берут кредиты. 
#посмотрим уникальные значения
unique_credit_purposes = df['purpose'].unique()
# print(unique_credit_purposes)
string = '; '.join(unique_credit_purposes)
lemmas = m.lemmatize(string)
from collections import Counter
display(Counter(lemmas))
#проверим сколько раз встречается каждый тип обращений
display(df['purpose'].value_counts())

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         '; ': 37,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

Чаще всего люди берут кредиты на недвижимость, автомобиль, образование 

### Категоризация данных

In [21]:
# сгруппирую стоблец по количеству детей
children_set = df.groupby('children')['debt'].sum() 
df.groupby('children')['debt'].count() 

children
0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: debt, dtype: int64

In [22]:
# Сгруппируем столбец дети по возрасту, чтобы убедиться в корректности данных
df[df['children'] == 20].groupby('dob_years')['dob_years'].count()
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [23]:
#Распределим клиентов так: клиенты младше 18 лет попадают в категорию «дети», Клиенты от 19 до 64 лет — категория «взрослые»;
# Клиенты старше 65 лет принадлежат к категории «пенсионеры».
def age_group(age):
    if age < 18:
        return 'дети'
    if age <= 64:
        return 'взрослые'
    return 'пенсионеры'
display(age_group(14)) # проверим правильно ли работает функция

'дети'

In [24]:
#добавим столбец с группой по возрастам 
df['age_group'] = df['dob_years'].apply(age_group)
df.head(10) 
# посмотрим статистику по вохрастам
display(df['age_group'].value_counts()) 

взрослые      20458
пенсионеры      895
дети            101
Name: age_group, dtype: int64

In [25]:
# распределим заемщиков по группам дохода:
def income_status(total_income):
    if total_income <= 88946.5:
            return 'Низкий уровень дохода'
    if total_income <= 135716.0:
            return 'Средний уровень дохода'
    if total_income < 195751.5:
            return 'Высокий уровень дохода'
    return 'Сверхвысокий уровень дохода'
df['income_status'] = df['total_income'].apply(income_status)

In [26]:
#напишем функцию для категоризации целей кредита 
def purpose_of(purpose):
    if 'образов' in purpose:
        return 'Образование'
    if 'авто' in purpose:
         return 'Автомобиль'
    if 'свад' in purpose:
         return 'Свадьба'
    if 'жил' or 'недв' in purpose:
        return 'Недвижимость'
    return 'некласифицированная цель'
df['purpose_of'] = df['purpose'].apply(purpose_of)

In [27]:
#посчитаем сколько целей всего получилось 
df['purpose_of'].unique()

array(['Недвижимость', 'Автомобиль', 'Образование', 'Свадьба'],
      dtype=object)

In [28]:
df['purpose_of'].value_counts()


Недвижимость    10811
Автомобиль       4306
Образование      4013
Свадьба          2324
Name: purpose_of, dtype: int64

In [29]:
#  словарь из двух списков: cсначала смотрю количество уникальных значений в столбце education_id
display(len(df['education_id'].unique()))

5

In [30]:
education_log = df[['education_id','education']]
display(education_log.head(10)) 

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


In [31]:
#создадим словарь:
education_dict = df[['education_id', 'education']]
display(education_dict.head(10)) 
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
display(education_dict.head()) 
display(education_log.groupby('education').mean().sort_values('education_id',ascending=False).head(10))

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,education_id
education,
ученая степень,4
начальное,3
неоконченное высшее,2
среднее,1
высшее,0


In [32]:
#проделаем тоже самое в фэмили статус
display(len(df['family_status_id'].unique()))# всего 5 категорий
family_log = df[['family_status_id','family_status']]
display(family_log.head(10))

5

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
5,1,гражданский брак
6,0,женат / замужем
7,0,женат / замужем
8,1,гражданский брак
9,0,женат / замужем


In [33]:
#создадим словарь:
family_dict = df[['family_status_id', 'family_status']]
display(family_dict.head(10)) 
family_dict = family_dict.drop_duplicates().reset_index(drop=True)
display(family_dict.head()) 
display(family_log.groupby('family_status').mean().sort_values('family_status_id',ascending=False).head(10))

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
5,1,гражданский брак
6,0,женат / замужем
7,0,женат / замужем
8,1,гражданский брак
9,0,женат / замужем


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


,family_status_id
family_status,
не женат / не замужем,4
в разводе,3
вдовец / вдова,2
гражданский брак,1
женат / замужем,0


В банк за кредитом обратились не только платежеспособные клиенты, но и дети и пенсионеры, для которых возможно кредитоспособность рассчитыватеся по другому
Кредиты берут в основном на четыре цели: недвижимость, авто, образование, свадьбу

## Ответы на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [34]:
children_set = df.groupby('children')['debt'].count()
children_pivot = df.pivot_table(index = ['children'], \
                                columns = ['debt'], \
                                values = 'purpose', aggfunc='count')
children_total = df['children'].value_counts() 
children_pivot['ratio'] = children_pivot[1] / children_total *100

display(children_pivot)

debt,0,1,ratio
children,,,
0,13028.0,1063.0,7.543822
1,4410.0,445.0,9.165808
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


Есть зависимость между наличием детей и возвратом кредита в срок.
Судя по высчитанному рейтингу семьи с четырьмя детьми закрывают кредиты нерегулярно. Самые надежные плательщики - бездетные семьи.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
# теперь посмотрим тоже самое с колонкой семейный статус 
family_status_set = df.groupby('family_status')['debt'].sum() / df.groupby('family_status')['debt'].count() * 100
family_status_set
family_status_pivot = df.pivot_table(index=['family_status'], columns=['debt'], values='total_income', aggfunc='count')
family_status_pivot['ratio'] = family_status_pivot[1] / family_status_pivot[0] * 100
family_status_pivot

debt,0,1,ratio
family_status,,,
в разводе,1110,85,7.657658
вдовец / вдова,896,63,7.031250
гражданский брак,3763,388,10.310922
женат / замужем,11408,931,8.160940
не женат / не замужем,2536,274,10.804416


Есть зависииость между семейным положением и выплатой кредита в срок.
Хуже всего отдают кредиты одинокие люди. Так как мы рассматриваем зависимость от количества детей, видимо % возврата зависит сколько в семье кормильцев. 
Самыми надежными плательщиками являются вдовцы и люди, состоящие в разводе

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
total_income_set = df.groupby('income_status')['debt'].sum() / df.groupby('income_status')['debt'].count() * 100
total_income_set.sort_values() # смотрим количество возвратов по уровням дохода
total_income_pivot = df.pivot_table(index=['income_status'], columns=['debt'], values='education', aggfunc='count')
total_income_pivot['ratio'] = total_income_pivot[1] / total_income_pivot[0] * 100
total_income_pivot


debt,0,1,ratio
income_status,,,
Высокий уровень дохода,6464,619,9.576114
Низкий уровень дохода,2999,249,8.302768
Сверхвысокий уровень дохода,4986,383,7.681508
Средний уровень дохода,5264,490,9.308511


Есть зависимость Между уровнем дохода и ваозвратом кредита в срок. Лучше всего возвращают кредиты люди в сверхвысоким уровнем дохода.Потому как чаще всего у них доход стабильный.
Затем люди с низким уровнем дохода. Возможно боятся накопления % за просрочки.
Потому как люди с высоким уровнем дохода имеют самый высокий процент невозврата кредита в срок.

### Как разные цели кредита влияют на его возврат в срок?

In [37]:
credit_purpose_set = df.groupby('purpose_of')['debt'].sum() / df.groupby('purpose_of')['debt'].count() * 100
credit_purpose_set.sort_values #количество возвратов в зависимости от цели 
credit_purpose_pivot = df.pivot_table(index=['purpose_of'], columns=['debt'], values='education', aggfunc='count')
credit_purpose_pivot['ratio'] = credit_purpose_pivot[1] / credit_purpose_pivot[0] * 100
credit_purpose_pivot


debt,0,1,ratio
purpose_of,,,
Автомобиль,3903,403,10.325391
Недвижимость,10029,782,7.797388
Образование,3643,370,10.156464
Свадьба,2138,186,8.699719


Цели кредита влияют на % его возврата.
Лучше всего возвращают кредиты на недвижимость и свадьбу. Возможно из-за размера кредитов и их долгосрочности. Срок ипотеки большой и легче рассчитать бюджет. Хужу всего возвращают кредиты на автомобиль.

# Общий вывод

1. уделить внимание качеству вводимой информации. 
Блокировать введнеие отрицательрых значений. Также внимательно отнестись к заведомо неточной информации - сверхвысоким показателям в стаже и в количестве детей. Возможно стоит перевести измерения стажа из дней в годы.
2. Проделана большая работа по выявлению целей кредита. Рекомендуется изначально разбить цели кредита на категории по унифицированной форме.
3. В ходе исследования выявлены основные цели кредитования: недвижимость, автомобиль, образование и свадьба.
4. В данных выявлены 4 группы возрастов. Рекомендуется удалить несовершеннолетних, а также, возможно создать отдельные данные по группам с особыми условиями кредитования (например, пенсионеры)
5. Семьи с 4 детьми, также как и одинокие люди, не находящиеся в браке находятся в группе риска по незакрытию кредитов. Стоит это учесть при оформлении и рассчитать риски и условия.
6. Рассмотреть условия автокредитования. Именно это ниша лидер по просрочкам и платежам по кредитам




